<a href="https://colab.research.google.com/github/ravik04/Real-Time-Public-Sentiment-Dashboard-Twitter-X-/blob/main/Real_Time_Public_Sentiment_Dashboard_(Twitter_X)_Ravi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tweepy nltk

In [ ]:
import tweepy
import time

bearer_token = "AAAAAAAAAAAAAAAAAAAAAMw21wEAAAAAC%2BNE00Bwvc2agGF7heh13zJ3THU%3D4clx3F7u4QpSbB5tJblDUKkaJgrGRmbx2OwEE7p7EuinwiAMGu"

client = tweepy.Client(bearer_token=bearer_token)

user_input = input("Enter a keyword to search for: ")

# Search for recent tweets mentioning a keyword (last 7 days)
query = f"{user_input} -is:retweet lang:en"

time.sleep(10)

try:
    response = client.search_recent_tweets(query=query, max_results=100, tweet_fields=['created_at', 'text'])
except tweepy.TooManyRequests:
    print("Rate limit hit. Waiting 15 minutes...")
    time.sleep(15 * 60)  # sleep 15 minutes


if response.data:
    for tweet in response.data:
        print(f"{tweet.created_at} - {tweet.text}")
else:
    print("No tweets found.")


In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the VADER sentiment lexicon
nltk.download('vader_lexicon')

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
# Utility functions for cleaning and sentiment analysis
def clean_tweet(text):
    return text.replace('\n', ' ').strip()

def analyze_sentiment(text):
    score = sia.polarity_scores(text)['compound']
    if score >= 0.05:
        return 'Positive', score
    elif score <= -0.05:
        return 'Negative', score
    else:
        return 'Neutral', score


In [ ]:
import pandas as pd

tweets_data = []

if response.data:
    for tweet in response.data:
        cleaned = clean_tweet(tweet.text)
        sentiment, score = analyze_sentiment(cleaned)
        tweets_data.append({
            'id': tweet.id,
            'Time': tweet.created_at,
            'Tweet': cleaned,
            'Sentiment': sentiment,
            'Score': score
        })

df = pd.DataFrame(tweets_data)
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv("tweet_sentiment_output.csv", index=False)
print("Data exported to tweet_sentiment_output.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot sentiment distribution
sns.countplot(data=df, x='Sentiment', palette='Set2')
plt.title(f"Sentiment Distribution for '{user_input}'")
plt.xlabel("Sentiment")
plt.ylabel("Number of Tweets")
plt.show()

# Optional: Time-series plot of sentiment scores
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values('Time', inplace=True)

plt.figure(figsize=(12,6))
sns.lineplot(data=df, x='Time', y='Score', label='Sentiment Score')
plt.title(f"Sentiment Score Trend for '{user_input}'")
plt.xlabel("Time")
plt.ylabel("Compound Sentiment Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import os

filename = 'daily_sentiment_log.csv'
file_exists = os.path.isfile(filename)

df.to_csv(filename, mode='a', header=not file_exists, index=False)
print(f"Data appended to {filename}")